In [1]:
import pandas as pd
import numpy as np
import patsy

from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV, ElasticNet, ElasticNetCV
from sklearn.model_selection import cross_validate, cross_val_score, cross_val_predict
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

In [2]:
kobe = pd.read_csv('/Users/paxton615/GA/resource-datasets/kobe_bryant/kobe_superwide_games.csv')
df = kobe.copy()

In [3]:
kobe.shape

(1558, 645)

In [4]:
kobe.columns

Index(['SHOTS_MADE', 'AWAY_GAME', 'SEASON_OPPONENT:atl:1996-97',
       'SEASON_OPPONENT:atl:1997-98', 'SEASON_OPPONENT:atl:1999-00',
       'SEASON_OPPONENT:atl:2000-01', 'SEASON_OPPONENT:atl:2001-02',
       'SEASON_OPPONENT:atl:2002-03', 'SEASON_OPPONENT:atl:2003-04',
       'SEASON_OPPONENT:atl:2004-05',
       ...
       'ACTION_TYPE:tip_layup_shot', 'ACTION_TYPE:tip_shot',
       'ACTION_TYPE:turnaround_bank_shot',
       'ACTION_TYPE:turnaround_fadeaway_bank_jump_shot',
       'ACTION_TYPE:turnaround_fadeaway_shot',
       'ACTION_TYPE:turnaround_finger_roll_shot',
       'ACTION_TYPE:turnaround_hook_shot', 'ACTION_TYPE:turnaround_jump_shot',
       'SEASON_GAME_NUMBER', 'CAREER_GAME_NUMBER'],
      dtype='object', length=645)

In [5]:
kobe.describe().T

,count,mean,std,min,25%,50%,75%,max
SHOTS_MADE,1558.0,7.358793,3.471180,0.0,5.00,7.0,10.000000,22.000000
AWAY_GAME,1558.0,0.500642,0.500160,0.0,0.00,1.0,1.000000,1.000000
SEASON_OPPONENT:atl:1996-97,1558.0,0.001284,0.035817,0.0,0.00,0.0,0.000000,1.000000
SEASON_OPPONENT:atl:1997-98,1558.0,0.001284,0.035817,0.0,0.00,0.0,0.000000,1.000000
SEASON_OPPONENT:atl:1999-00,1558.0,0.000642,0.025335,0.0,0.00,0.0,0.000000,1.000000
...,...,...,...,...,...,...,...,...
ACTION_TYPE:turnaround_finger_roll_shot,1558.0,0.000050,0.001390,0.0,0.00,0.0,0.000000,0.041667
ACTION_TYPE:turnaround_hook_shot,1558.0,0.000433,0.004902,0.0,0.00,0.0,0.000000,0.111111
ACTION_TYPE:turnaround_jump_shot,1558.0,0.031766,0.049402,0.0,0.00,0.0,0.052632,0.352941
SEASON_GAME_NUMBER,1558.0,42.946727,26.048206,1.0,21.00,42.0,63.000000,105.000000


- The columns are various statistics for each game. 
- There is a column SHOTS_MADE that will be our target variable for prediction
- This is good for regularization because there are so many columns (feature selection) and many of the columns represent similar things (multicollinearity)

#### make predictors and target variables. Standardize the predictors.

In [6]:
y = kobe['SHOTS_MADE']
X = kobe.iloc[:,1:] #包前不包后

In [7]:
scaler = StandardScaler()
Xs = scaler.fit_transform(X)
print('Mean:', Xs.mean(), 'Std:', Xs.std()) #依次检验scaler时候工作良好

Mean: 2.166996997490319e-18 Std: 1.0


##### SLR model

In [8]:
lr = LinearRegression()
lr.fit(Xs, y)

LinearRegression()

In [9]:
print(lr.intercept_, lr.coef_[:20])

7.360340622289268 [-1.51346980e-01 -3.07592285e+12  2.15723268e+11 -3.43957193e+12
 -1.02512161e+12  2.33524229e+12  1.69528345e+12  6.59243828e+11
 -4.84182866e+12 -2.78953002e+12  3.18939893e+11  3.70986277e+12
  1.54846776e+12 -4.04561219e+12 -5.80059422e+12 -8.20666497e+11
  4.86155175e+12 -1.25370422e+13  3.34899651e+12 -3.47743448e+12]


##### cross_val

In [10]:
linear = LinearRegression()
lin_scores = cross_val_score(linear, Xs, y, cv=5) #cv=5
print(lin_scores)
print(np.mean(lin_scores))

[-6.90282552e+28 -2.65458764e+28 -1.16402737e+28 -1.62255769e+28
 -1.77318843e+28]
-2.823437329833872e+28


In [11]:
linear = LinearRegression()
lin_scores = cross_val_score(linear, Xs, y, cv=10) #cv=10
print(lin_scores)
print(np.mean(lin_scores))

[-1.37331756e+29 -3.95516708e+27 -1.45305051e+28 -8.77477765e+27
 -2.74642208e+28 -3.31272689e+27 -4.17323855e+27 -5.04419267e+27
 -7.47214789e+27 -6.05918480e+28]
-2.7265058108827903e+28


#### Find the optimal alpha for Ridge

首先定义alph搜索的范围

In [17]:
ridge_alpha = np.logspace(0,5,200)

optimal_ridge = RidgeCV(alphas=ridge_alpha, cv=10)
optimal_ridge.fit(Xs, y)

print(optimal_ridge.alpha_)

775.2597488629465


In [18]:
optimal_ridge.score(Xs,y) # 效果比SLR好太多太多了！！！！！

0.7798565133622948

In [19]:
optimal_ridge.coef_.shape

(644,)

In [20]:
lr.coef_.shape

(644,)

### It's vastly better than the Linear Regression. There is likely so much multicollinearity in the data that "vanilla" regression overfits and has bogus coefficients on predictors. Ridge is able to manage the multicollinearity and get a good out-of-sample result.